In [1]:
# Useful imports
%matplotlib inline

import numpy as np
import libstempo.toasim as LT
import libstempo.plot as LP
from matplotlib.pyplot import *

from astropy.coordinates import SkyCoord

In addition to the stochastic background, we expect a number of closer, louder sources to stand out to us. These are high-mass, inspiraling black hole binaries with orbital frequencies around $10^{-8}$ Hz. Their orbital frequency evolves too fast for the binary to contribute significantly to the stochastic background, but, relative to a typical human lifespan, it evolves very little. As such, we term these signals "continuous gravitational waves" since PTA experiments don't witness these binaries changing in orbital frequency. 

Some of these potential CW sources are close and bright enough to be observed electromagnetically. Optical and X-ray observations of active galactic nuclei and quasars can indicate if a galaxy has a black hole at the center large enough for us to detect. We can use this supplemental astronomical data to inform our searches, giving the search pipeline a heads up about where a signal might be lurking. The information goes both ways, too. By achieveing upper limits from our latest dataset we've been able to make statements about the kinds of binaries which are definitely not present in our data, which in turn constrains the mass, frequency, etc. of the candidate binaries astronomers are also interested in.

In this exercise, we begin by looking at a continuous wave signal injected into some simulated pulsar timing data. First, we inspect the residuals by eye and do some quick estimates to give us an idea of how the injected signal gets jumbled. And then we run a full MCMC to see how much more information we can recover. Lastly, our results will be used to estimate some properties of the binary and it's host galaxy.

### 0.0 Make personalized directory 

In [2]:
INIT = #initials here
DIR = './cw_test'.format(INIT)
!mkdir './cw_test'.format(INIT)

SyntaxError: invalid syntax (<ipython-input-2-3e9052183b4f>, line 1)

# 1.0 Simulate Pulsar Data
### 1.1 Generate fake `.par` files

In [ ]:
# this function creates a parameter file for a simulated pulsar
# it randomizes sky location, name, pulse frequency and parallax

def make_fake_pulsar(DIR):
    '''
    Makes a fake pulsar par file
    '''
    output = "MODE 1\n"
    
    # Sphere Point Picking
    u = np.random.uniform()
    v = np.random.uniform()
    phi = 2*np.pi*u #using standard physics notation
    theta = np.arccos(2*v-1) - np.pi/2

    c = SkyCoord(phi,theta,frame='icrs',unit='rad')
    cstr = c.to_string('hmsdms')
    #print cstr
    RAJ = cstr.split(" ")[0].replace("h",":").replace("m",":")[:-1]
    DECJ = cstr.split(" ")[1].replace("d",":").replace("m",":")[:-1]
    cstr = cstr.replace(" ","")
    name = "J"+RAJ[0:2]+RAJ[3:5]+DECJ[0]+DECJ[1:3]+DECJ[4:6]

    output += "PSR      %s\n"%name

    
    output += "PEPOCH   50000.0\n"    
    output += "POSEPOCH   50000.0\n"

    period = 0.001*np.random.uniform(1,10) #seconds
    output += "F0       %0.10f 1\n"%(1.0/period)

    output += "RAJ      %s 1\n"%RAJ
    output += "DECJ     %s 1\n"%DECJ

    dist = np.random.uniform(0.1,5) #kpc
    output += "PX       %0.5f 1\n"%(1.0/dist) #mas

    filename = "%s/%s.par"%(DIR,name)
    print(filename)
    with open(filename,'w') as FILE:
        FILE.write(output)

    return filename.encode('ascii','ignore')

In [ ]:
par1 = make_fake_pulsar(DIR)
par2 = make_fake_pulsar(DIR)
par3 = make_fake_pulsar(DIR)

### 1.2  Generate fake `.tim` files with an injected single GW source


In [ ]:
# this function simulates a pulsar's timing file
# and injects the TOAs with a CW source of given chirp and frequency [Hz]

def observe(par,noise=0.5,mass=5e9,fgw=1e-8):
    ''' Noise in microseconds, mass in solar masses'''
    # let's set up some fake TOAs
    t = np.arange(53000,56650,30.0) #observing dates for 10 years
    t += np.random.randn(len(t)) #observe every 30+/-1 days
    psr = LT.fakepulsar(parfile=par,
                        obstimes=t,
                        toaerr=noise)
    LT.add_equad(psr,equad=noise*1e-6,seed=42) #this is mildly correlated white noise 
    
    # Let's add a source at the Virgo cluster: 12h27m +12d43'
    LT.add_cgw(psr, gwtheta=(12+43.0/60)*np.pi/180, gwphi=(12*15+27.0/60)*np.pi/180, mc=mass, dist=15, fgw=fgw, phase0=0, psi=0, inc=0, pdist=1.0,
               pphase=None, psrTerm=True, evolve=True,
               phase_approx=False, tref=0)
    psr.savetim('%s.tim'%par.split('.')[0])
    print('%s.tim'%par.split('.')[0])
    return psr

#Virgo cluster location in terms of celestial coordinates; useful for reading enterprise output
print((12+43.0/60)*np.pi/180,(12*15+27.0/60)*np.pi/180)

In [ ]:
psr1 = observe(par1.decode('utf-8'))
psr2 = observe(par2.decode('utf-8'))
psr3 = observe(par3.decode('utf-8'))

Let's take a look at the timing residuals with this injected signal with the `LP.plotres` function. Tip: if you are unsure how to use a particular imported function, you can write and execute `??<name of function>` to print a description of the function's input and output.

In [ ]:
# some space to work

## 2.0 Quick Estimates 

### 2.1 Approximate Gravitational Wave Strain

We can estimate the GW strain rather trivially from the timing perturbations. Conveniently, the root-mean-square (RMS) residuals are provided at the top of the libstempo plots. The strain $h \sim \Delta L/L$ is the fractional change in distance towards our pulsars. Therefore, we have

$$
\begin{eqnarray}
\Delta L & \sim & hL \sim hcT \\
& \approx & 9.5~\mathrm{m}~\left(\frac{h}{10^{-16}}\right)\left(\frac{T}{10~\mathrm{yr}}\right),
\end{eqnarray}
$$
where $L\sim cT$ is the length scale form the light/GW travel time. The timing precision is then simply
$$
\begin{eqnarray}
\Delta t & \sim & \frac{\Delta L}{c} \sim hT \\
& \approx & 32~\mathrm{ns}~\left(\frac{h}{10^{-16}}\right)\left(\frac{T}{10~\mathrm{yr}}\right).
\end{eqnarray}
$$

Using the RMS as the timing perturbation scale, what is your estimate of the GW strain?

In [ ]:
# some space to work 

## 3.0 Construct CW Signal Search

In [ ]:
# More imports for GW detection codes
import sys
# Enterprise
sys.path.append("/home/jovyan/work/shared/enterprise/")
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import enterprise_extensions.model_utils as utils
import enterprise_extensions.models as models


### 3.1 Add pulsars to model

In [ ]:
# Create enterprise Pulsar objects for each of your pulsars. You can make Pulsar objects with
#psr = Pulsar(<PARFILE>, <TIMFILE>, ephem="DE436", timing_code="tempo2")


# Here you should append your pulsar objects to this list for use by the model later
psrs = []

### 3.2 Form timing model

In [ ]:
# Create white noise parameter priors
efac = parameter.Constant(1)
equad = parameter.Uniform(-8.5,-5.0)

##### Signals below #####

# white noise parameters
ef = white_signals.MeasurementNoise(efac=efac)
eq = white_signals.EquadNoise(log10_equad=equad)

# timing model
tm = gp_signals.TimingModel()

### 3.3 Initialize CW source parameter priors & add to signal model

The strain amplitude is a function of a few of the source parameters,  

$$
\begin{equation}
h_{c} = \dfrac{2 \mathcal{M}_c^{5/3}(\pi f_{\rm GW})^{2/3}}{d_{L}}
\end{equation}
$$

(Note: This formula is an averaged over sky position.)

You'd anticipate that more massive systems are "louder," as are closer systems. This is reflected in the strain's dependence on the binary's effective mass, called the chrip mass ($\mathcal{M}_{c}$) and inverse dependence on a particular distance to the binary, called the luminosity distance ($d_L$).

If we have no prior knowledge about our continuous wave source, the only information we can recover from our search is about the strain (i.e., $h_{c}$). And since that is a mix of chirp mass and luminosity distance, we need some other information in order to get estimates of these two values independently. Fortunately, there is a way to untangle these values: through the inclusion of the so-called pulsar term.

The full influence of a gravitational wave passing through our pulsar array is a combination of the measurements of the signal at the Earth and at each pulsar individually. If the wave jiggles Earth, then the times of arrival for all the pulsars in the array will change. We encapsulate this particular portion of the signal effect into the "Earth-term". But, every pulsar experiences a jiggle too, and since the pulsar's pulses take time to reach us, we essentially see the signal in the past, before it reached Earth. This is the "pulsar-term" in our residuals. In combination, we uncover the time evolution of the signal. 

$$
\begin{equation}
	\Phi(t) = \Phi_0 + \Phi_p + \frac{2\pi}{32} \mathcal{M}^{-5/3} \left[ f(t_{p,0})^{-5/3} - f(t_p)^{-5/3} \right]
\end{equation}
$$

$$
\begin{equation}
	\Phi_p = \frac{2 \pi}{32} \mathcal{M}^{-5/3} \left[ f_0^{-5/3} - f(t_{p,0})^{-5/3} \right]
\end{equation}
$$

Theoretically, we understand this to provide information on the chirp mass of the system. By including both Earth- and pulsar-terms into our search, we're better able to recover the source properties. 


In [ ]:
# continuous GW parameters
# note that we are pre-initializing them with names here so that they will be shared
# across all pulsars in the PTA

# Our standard CW search looks for a GW with a specific frequency and we hold log10_fgw constant,
# as commented below. However, we will do a search in frequency
#freq = 8e-09
#log10_fgw = parameter.Constant(np.log10(freq))('log10_fgw')


cos_gwtheta = parameter.Uniform(-1, 1)('cos_gwtheta') #position of source
gwphi = parameter.Uniform(0, 2*np.pi)('gwphi') #position of source
log10_mc = parameter.Uniform(7, 10)('log10_mc') #chirp mass of binary
log10_fgw = parameter.Uniform(-10,-7)('log10_fgw') #gw frequency 
phase0 = parameter.Uniform(0, 2*np.pi)('phase0') #gw phase
psi = parameter.Uniform(0, np.pi)('psi') #gw polarization 
cos_inc = parameter.Uniform(-1, 1)('cos_inc') #inclination of binary with respect to Earth 

log10_h = parameter.LinearExp(-18, -11)('log10_h') #gw strain (linear exponential for an upper limit calculation)


# define CGW waveform and signal
cw_wf = models.cw_delay(cos_gwtheta=cos_gwtheta, gwphi=gwphi, log10_mc=log10_mc, 
                     log10_h=log10_h, log10_fgw=log10_fgw, phase0=phase0, 
                     psi=psi, cos_inc=cos_inc)
cw = models.CWSignal(cw_wf, ecc=False, psrTerm=True)

### 3.4 Define the full model (includes noise + GW signals)

In [ ]:
# full signal
s = ef + eq + tm + cw

### 3.5 With model and pulsars, create `enterprise` PTA object

In [ ]:
# initialize PTA
model = [s(psr) for psr in psrs]
pta = signal_base.PTA(model)

### 3.6 Prepare the MCMC sampler

In [ ]:
# Prepare sampler initial condition
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.1**2)

# parameter groupings
groups = utils.get_parameter_groups(pta)

# define where you want to put the chains from the MCMC
chaindir = 'chains/'

# set up jump groups by red noise groups (need better way of doing this)
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, groups=groups, 
                 outDir=chaindir, resume=False)

# write parameter file for convenience
filename = chaindir + '/params.txt'
np.savetxt(filename,list(map(str, pta.param_names)), fmt='%s')

In [ ]:
# add prior draws to proposal cycle -- this helps prevent the sampler's walkers 
# from getting trapped in local minimum in parameter space
jp = utils.JumpProposal(pta)
sampler.addProposalToCycle(jp.draw_from_prior, 5)
sampler.addProposalToCycle(jp.draw_from_cw_log_uniform_distribution, 10)

### 3.7 Sample!

In [ ]:
N = 100000
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50)

## 4.0 Visualize Results

In [ ]:
# Load the MCMC chains and parameter names (if you need them)

chain = np.loadtxt(chaindir + '/chain_1.txt')
params = list(np.loadtxt(chaindir + '/params.txt', dtype='str'))

# Value of burn-in to apply to the chains.
burn = int(0.25*chain.shape[0])

In [ ]:
# Convenience function to help plot the marginalized parameter distributions
def plot_param(name):
    '''Given one of the CGW function names above, plot the distribution'''
    hist(chain[burn:,params.index(name)], 50, normed=True, lw=2, color='C0', histtype='step')
    if "log10" in name:
        xlabel(r"$\log_{10} \mathrm{%s}$"%(name.split('_')[-1]))
    else:
        xlabel(name)

In [ ]:
# some space to work

In [ ]:
# plot
corner.corner(chain[burn:,:-5], labels=pars)

## 5.0 Post-Processing Analysis 

### 5.1 Upper and Lower Limits

The 11-year dataset was evaluated to find the 95% upper strain limit. This means with 95% confidence 95% of our strain samples lie below a value $h_{c,95}$. This can be transalted to a luminosity distance, and we can limit the mass of the SMBHB at the center of the Virgo cluster. 


From your analysis find the 95% upper limit of the strain amplitude and from that determine the 95% lower limit on the luminosity distance to the Virgo cluster.

For reference, the chirp mass for two equal mass black holes $m_1 = m_2 = M/2$  is

$$\mathcal{M}_c = \frac{(m_1 m_2)^{3/5}}{(m_1+m_2)^{1/5}} = \frac{(M/2)^{6/5}}{M^{1/5}} = 2^{-6/5} M \approx 0.435 M.$$


In [ ]:
from astropy.constants import M_sun, G, c
from astropy import units as u

def d_luminosity(h, M, f):
    """
    Eq. 17 from 5-yr paper, luminosity distance given 
    gravitational wave strain, f in Hz, M in solar masses, 
    and h (not log10(h)).
    
    :param h: characteristic strain
    :param M: binary chirp mass, units of solar masses
    :param f: search frequency, units of Hz
    
    :returns: 95% lower limit on luminosity distance in Mpc
    """
    d = (2 * (M * M_sun * G)**(5/3) * (np.pi * f * u.s**(-1))**(2/3) / (h * c**4)).to(u.Mpc)
    return d

What is your 95% strain upper limit and 95% sky-averaged lower luminosty distance limit?

In [ ]:
# some space to work

### 5.2 Orbital Separation

As the two black holes have a large separation, we can assume the system is in the Newtonian limit and use the Keplerian orbital frequency to relate $\Omega^2 = GM/a^3$, where $M$ is the total mass of the system and $a$ is the semi-major axis. The gravitational wave frequency is twice the orbital frequency, and so

$$
\begin{eqnarray}
f & = & 2\left(\frac{\Omega}{2\pi}\right) \\
& = &\frac{1}{\pi} \left(\frac{GM}{a^3}\right)^{1/2} \\
& \approx & 200~\mathrm{nHz}~\left(\frac{M}{10^8~M_\odot}\right)^{1/2} \left(\frac{a}{\mathrm{mpc}}\right)^{-3/2}
\end{eqnarray}
$$

What is the separation of the system?


In [ ]:
# some space to work

### 5.3 Decay Timescale

For a more general derivation, let's assume we have two masses $m_1$ and $m_2$, with total mass $M = m_1 + m_2$ and reduced mass $\mu = m_1 m_2 /M$, in a circular orbit and separated by a semi-major axis $a$. From Peters (1964; Phys. Rev. 136, B1224), one can write down the change in the semi-major axis as

$$\frac{da}{dt} = -\frac{64}{5} \frac{G^3 \mu M^2}{c^5 a^3}.$$

The decay timescale is then 

$$t_{\rm gw} = \int_a^0 \frac{dt}{da'} da' = \int_0^a \frac{5}{64} \frac{c^5 a'^3}{G^3 \mu M^2} da' = \frac{5}{256} \frac{c^5 a^4}{G^3 \mu M^2}.$$

We can write this expression in terms of a mass ratio $q \equiv m_1/m_2 \le 1$. It will be useful to substitute $M = m_1+m_2 = m_2 (q+1)$. Therefore, our mass terms can be written out as

$$\mu M^2 = m_1 m_2 M = m_1 m_2^2 (q+1) = m_2^3 q (q+1) = M^3 \frac{q}{(q+1)^2}.$$

Therefore, our decay timescale becomes

$$t_{\rm gw} = \frac{5}{256}\frac{c^5 a^4}{G^3 M^3} \frac{(q+1)^2}{q}.$$

Defining the dimensionless quantity $q_r = q/(q+1)^2$, we can write the scaling relation as

$$t_{\rm gw} = 4.88\times 10^6~\mathrm{yrs}~\left(\frac{a}{10^3~R_s}\right)^4 \left(\frac{M}{10^8~M_\odot}\right)^{-3} q_r^{-1},$$

where the Schwarzschild Radius $R_s = 2GM/c^2$ and we use the same fiducial mass of $10^8~M_\odot$.

Given your parameters above, how long will it take the system to merge?
What kind of equal-mass binaries exhibit significant frequency evolution over a the course of a typical PTA experiment?

In [ ]:
# some space to work

### 5.4 Host Galaxy Parameters

McConnell and Ma (2013) found a variety of scaling relations between various properties of a galaxy and its black hole mass. Presumably, with a single black hole after merger, the galaxy should still fall close to these relations, and so the relations should hold before the merger as well. The relations they find are

The Black Hole Mass-Stellar Velocity Dispersion ($M_{\mathrm{BH}}$-$\sigma$) relation:

$$\log_{10}(M_{\mathrm{BH}}) = 8.32 + 5.64 \log_{10}\left(\frac{\sigma}{200~\mathrm{km~s^{-1}}}\right)$$

The Black Hole Mass-Luminosity (V-band) ($M_{\mathrm{BH}}$-$L_V$) relation:

$$\log_{10}(M_{\mathrm{BH}}) = 9.23 + 1.11 \log_{10}\left(\frac{L_V}{10^{11}~L_\odot}\right)$$

The Black Hole Mass-Bulge Stellar Mass ($M_{\mathrm{BH}}$-$M_{\mathrm{bulge}}$) relation:

$$\log_{10}(M_{\mathrm{BH}}) = 8.46 + 1.05 \log_{10}\left(\frac{M_{\mathrm{bulge}}}{10^{11}~M_\odot}\right)$$

Given the more informative chirp mass estimate above, calculate values for the different characterisitics of the the host galaxy.

In [ ]:
# some space to work

## And beyond:

Now that you are an expert, try re-running the code in the following ways and see how your answers change:

* Reduce the simulated mass of the system in `observe()` to make the observed strain even weaker. How well can you do?
* Add red noise into the `observe()` function using `LT.add_rednoise(psr,A,gamma)`. $A$ will be in GW strain amplitude units and you can vary $\gamma$ in steepness, typically somewhere between 1 and 5. How does this affect the results? What new correlations do you see amongst the parameters? 

Remember: try adding more pulsars into your array if you need some additional signal boost!

In [ ]:
# some space to work